In [21]:
from openai import OpenAI
import json
import time
from tqdm import tqdm
from datetime import datetime
import re
import os

PRISONER_DILEMMA_PROMPT = """
You are facing a prisoner's dilemma:
- If both you and your partner stay silent, you each receive a short sentence.
- If you implicate your partner and they stay silent, you go free, and they receive a long sentence.
- If you stay silent while they implicate you, you receive a long sentence, and they go free.
- If you both implicate each other, you both receive a medium sentence.

Start your response with EXACTLY one word on the first line: either "Silent" or "Implicate".
Leave a blank line.
Then, provide your reasoning as one or more sentences.
What would you do?
"""

In [22]:
def parse_structured_response_openai(response_text):
    text = response_text.strip()
    if not text:
        return {
            "response_text": response_text,
            "decision": "NotJudged",
            "reason": "",
            "response": None
        }
    parts = text.split("\n\n", 1)
    decision_candidate = parts[0].strip().capitalize()
    if decision_candidate not in ["Silent", "Implicate"]:
        return {
            "response_text": response_text,
            "decision": "NotJudged",
            "reason": "",
            "response": None
        }
    reason = parts[1].strip() if len(parts) > 1 else ""
    is_silent = decision_candidate == "Silent"
    return {
        "response_text": response_text,
        "decision": decision_candidate,
        "reason": reason,
        "response": is_silent
    }

In [23]:
client = OpenAI(base_url="http://localhost:8000/v1", api_key="dummy")

print("Client set up. Make sure your vLLM/OpenRouter/OpenChatHub or similar server is running on port 8000!")

Client set up. Make sure your vLLM/OpenRouter/OpenChatHub or similar server is running on port 8000!


In [28]:
output_dir = "experiment_results_openai_local"
os.makedirs(output_dir, exist_ok=True)
output_prefix = os.path.join(output_dir, datetime.now().strftime("%Y%m%d_%H%M%S_openailocal_prisoner"))

iters = 1
modelname = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Or whatever your server exposes, e.g., "llama3.2:latest"
results = []

for i in tqdm(range(1, iters+1)):
    t0 = time.time()
    completion = client.chat.completions.create(
        model=modelname,
        messages=[
            {"role": "system", "content": PRISONER_DILEMMA_PROMPT},
            {"role": "user", "content": "What would you do?"}
        ],
        temperature=0.7,
        max_tokens=256
    )
    # Typical vLLM, OpenAI compatible API: extract content from first choice
    response_text = completion.choices[0].message.content.strip()
    t1 = time.time()
    parsed = parse_structured_response_openai(response_text)
    parsed.update({
        "iteration": i,
        "response_time": t1-t0,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })
    results.append(parsed)
    # Optional: print(parsed)

100%|██████████| 1/1 [00:03<00:00,  3.86s/it]


In [31]:
with open(f"{output_prefix}_openai_full.json", "w") as f:
    json.dump(results, f, indent=2)
print("Saved OpenAI localhost results to", f"{output_prefix}_openai_full.json")

print(json.dumps(results[0], indent=2))

Saved OpenAI localhost results to experiment_results_openai_local/20250421_150608_openailocal_prisoner_openai_full.json
{
  "response_text": "That would depend on the situation. If I had to choose, I would potentially implicate someone based on circumstantial evidence. In this scenario, my partner is implicating me, so I would likely stay silent and wait to see what happens. If the evidence is strong enough to implicate both of us, I would likely actively implicate my partner to protect myself. Ultimately, the best course of action would depend on the specific circumstances of the situation.",
  "decision": "NotJudged",
  "reason": "",
  "response": null,
  "iteration": 1,
  "response_time": 3.8545379638671875,
  "timestamp": "2025-04-21 15:06:12"
}
